In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Load PDF

In [2]:
file_path = "../example_data/PPPR 2024 - Overview.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

66


## Split

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

232

## Embed

In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v3",
    model_kwargs={
        "device":'cuda',
        "trust_remote_code": True
    })

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

In [5]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Vectors length: {len(vector_1)}\n")
print(vector_1[:10])

Vectors length: 1024

[0.0712890625, -0.154296875, -0.035888671875, -0.032470703125, -0.10400390625, -0.09228515625, -0.01904296875, 0.03125, -0.09130859375, -0.00628662109375]


## Vector stores

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

### Add the doc

In [7]:
ids = vector_store.add_documents(documents=all_splits)

### Try to query

In [8]:
results = vector_store.similarity_search_with_score("Timur Tengah")  # "Middle East" in Bahasa Indonesia

doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.5561332222845853

page_content='much slower rates than in other regions, and the number of people living in extreme poverty 
in the region has come fairly close to doubling—rising from 282 million in 1990 to 464 million 
in 2024. Similarly, in the Middle East and North Africa, the number of people living in extreme 
poverty doubled from 15 million in 1990 to 30 million in 2024. Extreme poverty in that region 
has surged since 2014, driven by fragility, conflict, and inflation (Gatti et al. 2023).' metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-10-04T09:08:57+05:30', 'author': 'The World Bank Group', 'moddate': '2024-10-04T09:18:42+05:30', 'title': 'The Poverty, Prosperity, and Planet Report 2024; Overview', 'trapped': '/False', 'source': '../example_data/PPPR 2024 - Overview.pdf', 'total_pages': 66, 'page': 21, 'page_label': '6', 'start_index': 3096}


### Use retrievers

In [9]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "Timur Tengah",
        "Asia Selatan",  # South Asia
    ],
)

[[Document(id='0b4ba9e8-e27d-4755-98b0-68e9b925404f', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-10-04T09:08:57+05:30', 'author': 'The World Bank Group', 'moddate': '2024-10-04T09:18:42+05:30', 'title': 'The Poverty, Prosperity, and Planet Report 2024; Overview', 'trapped': '/False', 'source': '../example_data/PPPR 2024 - Overview.pdf', 'total_pages': 66, 'page': 21, 'page_label': '6', 'start_index': 3096}, page_content='much slower rates than in other regions, and the number of people living in extreme poverty \nin the region has come fairly close to doubling—rising from 282 million in 1990 to 464 million \nin 2024. Similarly, in the Middle East and North Africa, the number of people living in extreme \npoverty doubled from 15 million in 1990 to 30 million in 2024. Extreme poverty in that region \nhas surged since 2014, driven by fragility, conflict, and inflation (Gatti et al. 2023).')],
 [Document(id='3c84a68f-